# OCR_certificate
Оптическое распознование символов (OCR) для справок формы 405.  
## Исходные данные
Имеются файлы изображения справок формата '.jpg' и соответствующие им файлы данных '.csv'.
## Задачи:
- Детектировать табличную часть справки и распознать текст.
- На выходе получить таблицу заданного вида.
- Оценить качество решения.
- Обернуть реализацию в микросервис.

In [27]:
import os

import pandas as pd

import numpy as np
import cv2 as cv
from PIL import Image
from PIL import ImageEnhance

import utils
from table import Table

import matplotlib.pyplot as plt

import pytesseract

import math

# 1. Первичный анализ и предобработка данных


## 1.1 Чтение всех файлов изображений и данных

Получим списки файлов '.jpg' и '.csv'.

In [16]:
file_dir="datasets/"
content = os.listdir(file_dir)
file_images=[]
file_csv=[]
for file in content:
    if os.path.isfile(os.path.join(file_dir,file)) and file.endswith('.jpg'):
        file_images.append(file)
    if os.path.isfile(os.path.join(file_dir,file)) and file.endswith('.csv'):
        file_csv.append(file)
print(len(file_images),len(file_csv))

16 15


Число фалов изображений не совпадает с числом файлов данных.  
Найдем файл которому нету пары или несколько таких файлов.

In [17]:
print(file_images[0],file_csv[0])

141899 .jpg 141899.csv


In [22]:
file_images_error = []
for f in file_images:
    f_number = f[:6]
    if f_number+'.csv' not in file_csv:
        file_images_error.append(f)
        print(f)

240493.jpg


Удалим название данного файла из списка

In [23]:
for f in file_images_error:
    file_images.remove(f)
len(file_images)

15

### 1.1.1 Прочитаем изображения в отдельный массив и создадим его копию.

In [38]:
imgs = []
for i in file_images:
    imgs.append(cv.imread(file_dir+i))
first_imgs = imgs.copy()

Рассмотрим изображение первой справки для понимания структуры.

In [35]:
cv.imshow("First", first_imgs[0])
cv.waitKey(0)

### 1.1.2 Прочитаем данные в отдельный массив.

Для понимания структуры файла данных рассмотрим первый из них.

In [32]:
data_first = pd.read_csv(file_dir+file_csv[1])
data_first.head()

,ID,ID пользователя,Класс крови,Дата донации,Дата добавления донации,Тип донации,Регион,Место стадчи,Статус донации,Есть справка
0,170415,204119,Цельная кровь,01.04.2015,23.12.2022,Безвозмездно,"Россия, Тульская область, Тула",556,Принята,Да
1,170430,204119,Цельная кровь,27.07.2017,23.12.2022,Безвозмездно,"Россия, Тульская область, Кимовск",Выездная акция,Принята,Да
2,170431,204119,Цельная кровь,11.10.2017,23.12.2022,Безвозмездно,"Россия, Тульская область, Кимовск",Выездная акция,Принята,Да
3,170432,204119,Цельная кровь,30.07.2018,23.12.2022,Безвозмездно,"Россия, Тульская область, Кимовск",Выездная акция,Принята,Да
4,170433,204119,Цельная кровь,16.10.2018,23.12.2022,Безвозмездно,"Россия, Тульская область, Кимовск",Выездная акция,Принята,Да


Из представленных данных в таблице изображения имеются столбцы 'Класс крови', 'Дата донации' , 'Тип донации' в результирующей таблице оставим только их.

In [43]:
data_csv = []
for f in file_csv:
    data_csv.append(pd.read_csv(file_dir+f)[['Класс крови','Дата донации','Тип донации']])

Проверим что полученные таблицы содержат по три столбца.

In [48]:
for i in range(len(data_csv)):
    if data_csv[i].shape[1] != 3:
        print('Ошибка для таблицы №',i)
    

Ошибки отсутствуют.  
Теперь найдем уникальные значения для каждого столбца с теустовыми значениями.

In [63]:
res = data_csv[0]
for i in range(1,len(data_csv)):
    res = pd.concat([res,data_csv[i]])
print('Для класса крови уникальными значениями являются:\n',res['Класс крови'].unique(),'\n'
      ,'Для типа донации уникальными значениями являются:\n',res['Тип донации'].unique(),'\n')

Для класса крови уникальными значениями являются:
 ['Цельная кровь' 'Плазма' 'Тромбоциты'] 
 Для типа донации уникальными значениями являются:
 ['Безвозмездно'] 



***Видно первый недостаток в предоставленных данных - отсутствие платной сдачи крови***

Рассмотрим как класс крови и тип донации записан в справках.

In [64]:
cv.imshow("First", first_imgs[0])
cv.waitKey(0)

-1

Там представлены два значения:  
1. кр/д(бв) - Цельная кровь, Безвозмездно  
2. пл/д(бв) - Плазма, Безвозмезно 

Необходимо найти изображение, где есть класс крови тромбоциты и просмотреть его. 

In [74]:
for i in range(len(data_csv)):
    if 'Тромбоциты' in data_csv[i]['Класс крови'].unique():
        print(i)

4
11


Рассмотрим четвертое изображение.

In [73]:
cv.imshow("First", first_imgs[4])
cv.waitKey(0)

-1

После просмотра получили дополнительную информацию, о существовании двух типов справок 405: 4я = 405-01/у и 0я = 405-05/у.  
В четвертой справке три типа записи:  
1. к/д(бв) - Цельная кровь, Безвозмездно  
2. п/ф(бв) - Плазма, Безвозмезно  
3. т/ф(бв) - Тромбоциты, Безвозмезно  

Рассмотрим одиннадцотое изображение.  

In [75]:
cv.imshow("First", first_imgs[11])
cv.waitKey(0)

-1

Там представлены два значения:  
1. кр/д(бв) - Цельная кровь, Безвозмездно  
2. пл/д(бв) - Плазма, Безвозмезно 
3. ц/д(бв) - Тромбоциты, Безвозмезно  

При просмотре донормативной документации не найдено сокращения для платной сдачи крови.  
Сделаем предположение, что начинается оно с буквы "п".  
Соответственно, можно создать два словаря соответствия.  

In [76]:
blood_class = {'к':'Цельная кровь','п':'Плазма','т':'Тромбоциты','ц':'Тромбоциты'}
donation_type = {'б':'Безвозмезно','п':'Платно'}

# 2. Распознование таблиц в изображениях